In [ ]:
import numpy as np
import sys
import os
import pandas as pd
sys.path.append('../../')

from script.Model_run import Speech_analysis

In [ ]:
#data_home_dir = '/home/kvattis/Documents/data/'
data_home_dir = '/home/kvattis/Dropbox (Partners HealthCare)/Data_Raw_Biogen_SCA/'

In [ ]:
speech_analyzer = Speech_analysis(print_on = False)

In [ ]:
output_biogen = pd.DataFrame(columns = ['Date','subject_id','Neuropheno_ID','Bars','Bars_Speech','Sex','Diagnosis','DoB','Samples_no','Prob_AT_tf_grad', 'Bars_Speech_pred_t','Bars_Total_pred_t', 'Bars_Speech_pred_tf','Bars_Total_pred_tf'])

# Biogen SCA study

In [ ]:
#survey_lex = pd.read_csv(data_home_dir + 'Biogen_SCA_Speech_Data/' + 'surveylex_questionIDs.csv',encoding='latin1')
survey_lex = pd.read_excel(data_home_dir + 'Biogen_SCA_Speech_Data/' + 'surveylex_questionIDs.xlsx',engine='openpyxl')
taskslist = survey_lex['question'].unique()

for tt in  taskslist: 
    if tt not in ["la_la_la","go_go_go","me_me_me", "lalala", "gogogo", "mememe","papapa"]:
        continue
    print(tt)
    pd.set_option('display.max_columns', 4)
    np.dtype(np.int16)
    task = str(tt)
    task_dir = str(tt)
    
    study_name = 'Biogen_SCA_Speech_Data'
    study_dir = data_home_dir + 'Biogen_SCA_Speech_Data/'
    #survey_lex = pd.read_csv(study_dir + 'surveylex_questionIDs.csv',encoding='latin1')
    survey_lex = pd.read_excel(study_dir + 'surveylex_questionIDs.xlsx',engine='openpyxl')

    survey_response_A = pd.read_excel(study_dir + 'surveyA-responses-summary_.xlsx',engine='openpyxl')
    survey_response_B = pd.read_excel(study_dir + 'surveyB-responses-summary_.xlsx',engine='openpyxl')
    survey_response_C = pd.read_excel(study_dir + 'surveyC-responses-summary_.xlsx',engine='openpyxl')
    survey_response_D = pd.read_excel(study_dir + 'surveyD-responses-summary_.xlsx',engine='openpyxl')
    
    survey_response = pd.concat([survey_response_A, survey_response_B, survey_response_C, survey_response_D], ignore_index=True)

    survey_submissions_A = pd.read_excel(study_dir + 'surveyA-submissions-summary_.xlsx',engine='openpyxl')
    survey_submissions_B = pd.read_excel(study_dir + 'surveyB-submissions-summary_.xlsx',engine='openpyxl')
    survey_submissions_C = pd.read_excel(study_dir + 'surveyC-submissions-summary_.xlsx',engine='openpyxl')
    survey_submissions_D = pd.read_excel(study_dir + 'surveyD-submissions-summary_.xlsx',engine='openpyxl')
    

    survey_submissions = pd.concat([survey_submissions_A, survey_submissions_B, survey_submissions_C, survey_submissions_D], ignore_index=True)

    #Patient_info = pd.read_csv(study_dir + 'Subjects_BiogenSCA_version2.csv',encoding='latin1')
    #Patient_info = pd.read_csv(data_home_dir + 'Subjects_BiogenSCA_version2.csv',encoding='latin1')
    #Patient_info_ = Patient_info.loc[:,['subject_id','neuropheno_id','diagnosis','bars_total_excl_miss_1','bars_total_excl_miss_2', 'bars_speech_1', 'bars_speech_2','sex', 'age']]
    
    _Patient_info = pd.read_csv(data_home_dir + 'Subjects_BiogenSCA_version2.csv',encoding='latin1')
    _Patient_info_ = _Patient_info.loc[:,['subject_id','neuropheno_id','diagnosis','bars_total_excl_miss_1','bars_total_excl_miss_2', 'bars_speech_1', 'bars_speech_2','sex']]

    Patient_info = pd.read_excel(data_home_dir + 'Subjects_BiogenSCA_with_PROM_2022_03_29_converted.xlsx',engine='openpyxl')
    Patient_info_ = Patient_info.loc[:,['subject_id','neuropheno_id','diagnosis','bars_total_excl_miss_1','bars_total_excl_miss_2', 'bars_speech_1', 'bars_speech_2','sex','dob']]
    
    for i in _Patient_info_['subject_id']:
        Patient_info_.loc[Patient_info_['subject_id'] == i,'neuropheno_id']  = _Patient_info_.loc[_Patient_info_['subject_id'] == i,'neuropheno_id']
    
    survey_lex_ = survey_lex.loc[:,['questionId','question']]
    survey_response_ = survey_response.loc[:,['sampleId','questionId', 'sessionId','createdDate']]
    survey_response_['createdDate'] = survey_response_['createdDate'].apply(lambda x: x[0:4] + '_' + x[5:7] + '_' + x[8:10] )
    survey_submissions_ = survey_submissions.loc[:,['sessionId','Your participant ID:']]
    survey_submissions_ = survey_submissions_.rename(columns={"Your participant ID:": "subject_id"})
    df = pd.merge(pd.merge(pd.merge(survey_response_,survey_lex_,on='questionId'), survey_submissions_, on = 'sessionId'),Patient_info_, on = 'subject_id')
    df.dropna(subset = ["sampleId"], inplace=True)

    df_task = df.loc[df['question'] == task][['sampleId','subject_id','neuropheno_id','diagnosis','createdDate','bars_total_excl_miss_1','bars_total_excl_miss_2', 'bars_speech_1', 'bars_speech_2','sex']]#, 'age']]
    df_task_numpy = df_task.to_numpy()

    
        
    list_diagnostic = ['Control', 'Ataxia', 'MSA', 'Other']

    
    #Going over all the files for the task
    list_names = []
    for entry in df_task_numpy:
        file_name = entry[0]
        ID = str(entry[1])

        if  not np.isnan(entry[2]):
            Neur_phen_id = str(int(entry[2]))
        else:
            Neur_phen_id = ID
        diagn = entry[3]
        date = entry[4]
        Date = date[:4] + date[5:7] + date[8:]
        bars_total = (entry[5] + entry[6])/2.
        bars_speech = (entry[7] + entry[8])/2./10.
        sex = entry[9]
        #age = entry[10]
        
        path_to_file = study_dir + ID + '_' + date + '_A/' + file_name + '.wav'
        path_to_file_flag = 0
        
        if os.path.exists(path_to_file):
            path_to_file_flag = 1 
        
        if not os.path.exists(path_to_file):
            path_to_file = study_dir + ID + '_' + date + '_B/' + file_name + '.wav'
        else:
            path_to_file_flag = 1
        
        if not os.path.exists(path_to_file):
            path_to_file = study_dir + ID + '_' + date + '_C/' + file_name + '.wav'
        else:
            path_to_file_flag = 1
            
        if not os.path.exists(path_to_file):
            path_to_file = study_dir + ID + '_' + date + '_D/' + file_name + '.wav'
        else:
            path_to_file_flag = 1    
            
        if path_to_file_flag == 0: continue;
            
        #Assign label
        if diagn == 0:
            #diagn = 'Control'
            label = 0
        elif diagn == 1:
            #diagn = 'Ataxia'
            label = 1
        elif diagn == 2:
            #diagn = 'Ataxia'
            label = 1
        elif diagn == 3:
            #diagn = 'Ataxia'
            label = 1
        elif diagn == 7:
            #diagn = 'MSA'
            label = 7
        else:
            #diagn = 'Other'
            label = 5
        
        if label not in [0,1]:
            continue
        
        if label == 0:
            bars_total = 0
            bars_speech = 0 
        
        if Neur_phen_id == ID:
            no_samples = speech_analyzer.load(path_to_file)
            Neur_phen_id = None
        else:
            no_samples = speech_analyzer.load(path_to_file,Neur_phen_id)
        
        if no_samples == 0: 
            continue
            
        output_biogen = output_biogen.append({'Date' : Date, 'subject_id' : ID, 'Neuropheno_ID' : Neur_phen_id, 'Bars': bars_total, 'Bars_Speech':bars_speech, 'Sex': sex, 'Diagnosis': diagn,'Samples_no' : speech_analyzer.sample_size_s, 'Prob_AT_tf_grad': speech_analyzer.classify(),
                               'Bars_Speech_pred_t': speech_analyzer.BARS_speech(), 'Bars_Total_pred_t': speech_analyzer.BARS_total(), 'Bars_Speech_pred_tf': speech_analyzer.BARS_speech_both_grad(), 'Bars_Total_pred_tf': speech_analyzer.BARS_total_both_grad()}, 
                ignore_index = True)
        

In [ ]:
output_biogen = output_biogen.astype(float).groupby(['Date','subject_id'], as_index = False).median().sort_values(by=['subject_id'])

for i in output_biogen['subject_id'].unique():
    output_biogen.loc[output_biogen['subject_id'] == i,'DoB'] = pd.to_datetime(Patient_info_.loc[Patient_info_['subject_id'] == i,'dob'].item())

output_biogen.to_csv('/home/kvattis/Desktop/Biogen_SCA_outputs_2022_05_04.csv', index = False)

# Phenotyping study

In [ ]:
output_phenotype =  pd.DataFrame(columns = ['Date','subject_id','Neuropheno_ID','Bars','Bars_Speech','Sex','Diagnosis','Age','Samples_no','Prob_AT_tf_grad', 'Bars_Speech_pred_t','Bars_Total_pred_t', 'Bars_Speech_pred_tf','Bars_Total_pred_tf'])
data_home_dir = '/home/kvattis/Documents/data/'
list_diagnostic = ['Control', 'Ataxia', 'PD', 'MG', 'FND', 'Other', 'Unknown']
#Read the Patient info csv and extract the diagnosis
Patient_info = pd.read_csv(data_home_dir + 'Audio_All/2021_01_27_All_Tasks-data.csv',encoding='latin1')
Patient_info['gen_diagnosis'] = Patient_info['gen_diagnosis'].replace(['AT'], 'Ataxia')
Patient_info['P_ID'] = Patient_info['ID'].apply(lambda x: x[0:5] )
Patient_info.set_index('ID', inplace = True)
Patient_info_ph = Patient_info
Gen_diagnosis = Patient_info['gen_diagnosis']

for task in ['Go1', 'Go2', 'La1', 'La2','Me1', 'Me2']:
    print(task)
    task_dir = data_home_dir + 'Audio_All/'+ task + '/' 
    audio_dir = task_dir + 'audio_chopped/'

    #Going over all the files for the task
    list_names = []
    
    for filename in os.listdir(audio_dir):
        if filename[-3:] != "wav":
            continue
        I_D = filename[0:16]
        Date = int(I_D[6:10] + I_D[11:13] + I_D[14:16])
        P_ID = filename[0:5]
        diagn = Gen_diagnosis.loc[I_D]
        Neur_phen_id = None
        path_to_file = audio_dir + filename
        
        #Assign label
        if diagn == 'Control':
            label = 0
        elif diagn == 'Ataxia':
            label = 1
        elif diagn == 'PD':
            label = 2
        elif diagn == 'MG':
            label = 3
        elif diagn == 'FND':
            label = 4
        elif diagn == 'Other':
            label = 5
        elif diagn == 'Unknown':
            label = 6
        
        if label not in [0,1]:
            continue
        
        bars_info = Patient_info[['bars_total', 'bars_arm_L', 'bars_arm_R', 'bars_leg_L', 'bars_leg_R', 'bars_oculomotor', 'bars_gait']].loc[str(I_D)]
        if bars_info.shape == (7,):
            bars_total = bars_info[0]
            bars_arm = bars_info[1] +  bars_info[2]
            bars_leg = bars_info[3] +  bars_info[4] 
            bars_occ = bars_info[5]
            bars_gait = bars_info[6]
            bars_speech = bars_total - (bars_arm + bars_leg + bars_occ + bars_gait)
        else:
            bars = bars_info[bars_info['ID'] == old_id]['bars_total'][0]
            bars_arm = bars_info[bars_info['ID'] == old_id]['bars_arm_L'][0] + bars_info[bars_info['ID'] == old_id]['bars_arm_R'][0] 
            bars_leg = bars_info[bars_info['ID'] == old_id]['bars_leg_L'][0] + bars_info[bars_info['ID'] == old_id]['bars_leg_R'][0] 
            bars_occ = bars_info[bars_info['ID'] == old_id]['bars_oculomotor'][0]
            bars_gait = bars_info[bars_info['ID'] == old_id]['bars_gait'][0]
            bars_speech = bars - (bars_arm + bars_leg + bars_occ + bars_gait)
            
        sex = Patient_info['sex'].loc[str(I_D)]
        sex =  1 if sex == 'F' else 2
        age = Patient_info['age'].loc[str(I_D)]
        
        no_samples = speech_analyzer.load(path_to_file,P_ID)
        if no_samples == 0: 
            continue
            
        output_phenotype = output_phenotype.append({'Date' : Date, 'subject_id' : P_ID, 'Neuropheno_ID' : Neur_phen_id,'Age': age, 'Bars': bars_total, 'Bars_Speech':bars_speech, 'Sex': sex, 'Diagnosis': label,'Samples_no' : speech_analyzer.sample_size_s, 'Prob_AT_tf_grad': speech_analyzer.classify(),
                               'Bars_Speech_pred_t': speech_analyzer.BARS_speech(), 'Bars_Total_pred_t': speech_analyzer.BARS_total(), 'Bars_Speech_pred_tf': speech_analyzer.BARS_speech_both_grad(), 'Bars_Total_pred_tf': speech_analyzer.BARS_total_both_grad()}, 
                ignore_index = True)

In [ ]:
output_phenotype = output_phenotype.astype(float).groupby(['Date','subject_id'], as_index = False).median().sort_values(by=['subject_id'])

output_phenotype.to_csv('/home/kvattis/Desktop/Phenotype_outputs_2022_05_04.csv', index = False)

# Remote AT study

In [ ]:
data_home_dir = '/home/kvattis/Documents/data/'
survey_lex = pd.read_excel(data_home_dir + 'Remote_AT_Speech_Data/' + 'surveylex_questionIDs.xlsx',engine='openpyxl')
taskslist = survey_lex['question'].unique()
output_remote =  pd.DataFrame(columns = ['Date','subject_id','Neuropheno_ID','Bars','Bars_Speech','Sex','Diagnosis','Age','Samples_no','Prob_AT_tf_grad', 'Bars_Speech_pred_t','Bars_Total_pred_t', 'Bars_Speech_pred_tf','Bars_Total_pred_tf'])

for tt in  taskslist: 
    if tt not in ["la_la_la","go_go_go","me_me_me", "lalala", "gogogo", "mememe","papapa"]: #['moy_forwards']:
        continue
    print(tt)
    pd.set_option('display.max_columns', 4)
    np.dtype(np.int16)
    
    task = str(tt)
    task_dir = str(tt)


    study_name = 'Remote_AT_Speech_Data'
    study_dir = data_home_dir + 'Remote_AT_Speech_Data/'
    survey_lex = pd.read_excel(study_dir + 'surveylex_questionIDs.xlsx',engine='openpyxl')

    survey_response_A_7 = pd.read_excel(study_dir + 'surveyA_7_responses_summary.xlsx',engine='openpyxl')
    survey_response_A_8_13 = pd.read_excel(study_dir + 'surveyA_8_13_responses_summary.xlsx',engine='openpyxl')
    survey_response_A_14 = pd.read_excel(study_dir + 'surveyA_14_responses_summary.xlsx',engine='openpyxl')
    survey_response_B_7 = pd.read_excel(study_dir + 'surveyB_7_responses_summary.xlsx',engine='openpyxl')
    survey_response_B_8_13 = pd.read_excel(study_dir + 'surveyB_8_13_responses_summary.xlsx',engine='openpyxl')
    survey_response_B_14 = pd.read_excel(study_dir + 'surveyB_14_responses_summary.xlsx',engine='openpyxl')
    survey_response_C_7 = pd.read_excel(study_dir + 'surveyC_7_responses_summary.xlsx',engine='openpyxl')
    survey_response_C_8_13 = pd.read_excel(study_dir + 'surveyC_8_13_responses_summary.xlsx',engine='openpyxl')
    #survey_response_C_14 = pd.read_excel(study_dir + 'surveyC_14_responses_summary.xlsx',engine='openpyxl')
    survey_response = pd.concat([survey_response_A_7, survey_response_A_8_13, survey_response_A_14, survey_response_B_7, survey_response_B_8_13, survey_response_B_14, survey_response_C_7, survey_response_C_8_13], ignore_index=True)

    survey_submissions_A_7 = pd.read_excel(study_dir + 'surveyA_7_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_A_8_13 = pd.read_excel(study_dir + 'surveyA_8_13_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_A_14 = pd.read_excel(study_dir + 'surveyA_14_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_B_7 = pd.read_excel(study_dir + 'surveyB_7_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_B_8_13 = pd.read_excel(study_dir + 'surveyB_8_13_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_B_14 = pd.read_excel(study_dir + 'surveyB_14_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_C_7 = pd.read_excel(study_dir + 'surveyC_7_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions_C_8_13 = pd.read_excel(study_dir + 'surveyC_8_13_submissions_summary.xlsx',engine='openpyxl')
    #survey_submissions_C_14 = pd.read_excel(study_dir + 'surveyC_14_submissions_summary.xlsx',engine='openpyxl')
    survey_submissions = pd.concat([survey_submissions_A_7, survey_submissions_A_8_13, survey_submissions_A_14, survey_submissions_B_7, survey_submissions_B_8_13, survey_submissions_B_14, survey_submissions_C_7, survey_submissions_C_8_13], ignore_index=True)

    Patient_info = pd.read_csv(study_dir + 'Subjects_Remote_Behavior.csv',encoding='latin1')
    Patient_info_ = Patient_info.loc[:,['subject_id','neuropheno_id','Diagnosis_1','Neuropheno_session_for_BARS','DOB']]
    Patient_info_['Diagnosis_1'] = Patient_info_['Diagnosis_1'].apply(lambda x: 0 if x == 'Control' else 1)
    Patient_info_ = Patient_info_.rename(columns={"Diagnosis_1": "diagnosis"})

    survey_lex_ = survey_lex.loc[:,['questionId','question']]
    survey_response_ = survey_response.loc[:,['sampleId','questionId', 'sessionId','createdDate']]
    survey_response_['createdDate'] = survey_response_['createdDate'].apply(lambda x: x[0:4] + '_' + x[5:7] + '_' + x[8:10] )
    survey_submissions_ = survey_submissions.loc[:,['sessionId','Your participant ID:']]
    survey_submissions_ = survey_submissions_.rename(columns={"Your participant ID:": "subject_id"})
    df = pd.merge(pd.merge(pd.merge(survey_response_,survey_lex_,on='questionId'), survey_submissions_, on = 'sessionId'),Patient_info_, on = 'subject_id')
    df.dropna(subset = ["sampleId"], inplace=True)
    df_task = df.loc[df['question'] == task][['sampleId','subject_id','neuropheno_id','diagnosis','createdDate','Neuropheno_session_for_BARS']]
    df_task_numpy = df_task.to_numpy()

    list_diagnostic = ['Control', 'Ataxia', 'MSA', 'Other']

    #Going over all the files for the task
    list_names = []
    for entry in df_task_numpy:
        file_name = entry[0]
        ID = str(entry[1])

        if  not np.isnan(entry[2]):
            Neur_phen_id = str(int(entry[2]))
            Neuropheno_session_for_BARS = str(entry[5])
        else:
            Neur_phen_id = ID
            Neuropheno_session_for_BARS = np.nan
        diagn = entry[3]
        date = entry[4]
        Date = date[:4] + date[5:7] + date[8:]
        
        if  str(Neuropheno_session_for_BARS) == 'nan':
            bars_total = np.nan
            bars_speech = np.nan
        else:
            bars_info = Patient_info_ph.loc[str(Neuropheno_session_for_BARS)]
            bars_total = bars_info['bars_total']
            bars_speech = bars_info['bars_total'] - (bars_info['bars_arm_L'] + bars_info['bars_arm_R'] + bars_info['bars_leg_L'] + bars_info['bars_leg_R'] + bars_info['bars_oculomotor'] + bars_info['bars_gait'])
        
        path_to_file = study_dir + ID + '_' + date + '/' + file_name + '.wav'

        if ID + '_' + date == '20023_2021_03_09':
            continue

        if ID + '_' + date == '20088_2021_03_14':
            continue

        #Assign label
        if diagn == 0:
            #diagn = 'Control'
            label = 0
        else:
            #diagn = 'Ataxia'
            label = 1
        
        
        if Neur_phen_id == ID:
            no_samples = speech_analyzer.load(path_to_file,ID)
            Neur_phen_id = None
        else:
            no_samples = speech_analyzer.load(path_to_file,Neur_phen_id)
        
        if no_samples == 0: 
            continue
            
        output_remote = output_remote.append({'Date' : Date, 'subject_id' : ID, 'Neuropheno_ID' : Neur_phen_id, 'Bars': bars_total, 'Bars_Speech':bars_speech, 'Sex': sex, 'Diagnosis': diagn,'Samples_no' : speech_analyzer.sample_size_s, 'Prob_AT_tf_grad': speech_analyzer.classify(),
                               'Bars_Speech_pred_t': speech_analyzer.BARS_speech(), 'Bars_Total_pred_t': speech_analyzer.BARS_total(), 'Bars_Speech_pred_tf': speech_analyzer.BARS_speech_both_grad(), 'Bars_Total_pred_tf': speech_analyzer.BARS_total_both_grad()}, 
                ignore_index = True)

In [ ]:
Patient_info = pd.read_csv(study_dir + 'Subjects_Remote_Behavior.csv',encoding='latin1')
Patient_info_ = Patient_info.loc[:,['subject_id','neuropheno_id','Diagnosis_1','Neuropheno_session_for_BARS','DOB']]
Patient_info_['Diagnosis_1'] = Patient_info_['Diagnosis_1'].apply(lambda x: 0 if x == 'Control' else 1)
Patient_info_ = Patient_info_.rename(columns={"Diagnosis_1": "diagnosis"})

In [ ]:
output_remote = output_remote.astype(float).groupby(['Date','subject_id'], as_index = False).median().sort_values(by=['subject_id'])

for i in output_remote['subject_id'].unique():
    output_remote.loc[output_remote['subject_id'] == i,'DoB'] = pd.to_datetime(Patient_info_.loc[Patient_info_['subject_id'] == i,'DOB'].item())

output_remote.to_csv('/home/kvattis/Desktop/AT_outputs_2022_05_04.csv', index = False)

In [ ]:
output_remote['Age']

In [ ]:
output_remote['Age'] = (pd.to_datetime(output_remote['Date'], format='%Y%m%d') - pd.to_datetime(output_remote['DoB']))/np.timedelta64(1,'Y')

In [ ]:
output_phenotype

In [ ]:
output_biogen['Date']

In [ ]:
output_biogen['Age'] = (pd.to_datetime(output_biogen['Date'], format='%Y%m%d') - pd.to_datetime(output_biogen['DoB']))/np.timedelta64(1,'Y')

In [ ]:
output_biogen['Age']

In [ ]:
output = pd.concat([output_phenotype,output_biogen,output_remote],ignore_index=True)

In [ ]:
output['Age']

In [ ]:
output.to_csv('/home/kvattis/Desktop/All_2022_05_04.csv', index = False)

In [ ]:
output.loc[output['Diagnosis']=='Ataxia','Diagnosis'] = 1

In [ ]:
output[output['Diagnosis']=='Ataxia']['Diagnosis']

In [ ]:
output.columns

In [ ]:
output = output.astype(float).groupby(['Date','subject_id'], as_index = False).median().sort_values(by=['subject_id'])

In [ ]:
speech_analyzer.plot_wav()

In [ ]:
speech_analyzer.play_audio_nr()

In [ ]:
speech_analyzer.play_resampled_audio_nr()

In [ ]:
speech_analyzer.plot_mel()

In [ ]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 200)
np.dtype(np.int16)

In [ ]:
output.tail(200)[['subject_id','Diagnosis']]

In [ ]:
Patient_info_at = Patient_info_
data_home_dir = '/home/kvattis/Dropbox (Partners HealthCare)/Data_Raw_Biogen_SCA/'
_Patient_info = pd.read_csv(data_home_dir + 'Subjects_BiogenSCA_version2.csv',encoding='latin1')
_Patient_info_ = _Patient_info.loc[:,['subject_id','neuropheno_id','diagnosis','bars_total_excl_miss_1','bars_total_excl_miss_2', 'bars_speech_1', 'bars_speech_2','sex']]

Patient_info_ = pd.read_excel(data_home_dir + 'Subjects_BiogenSCA_with_PROM_2022_03_29_converted.xlsx',engine='openpyxl')
Patient_info_ = Patient_info_.loc[:,['subject_id','neuropheno_id','diagnosis','bars_total_excl_miss_1','bars_total_excl_miss_2', 'bars_speech_1', 'bars_speech_2','sex','dob']]

for i in _Patient_info_['subject_id']:
    Patient_info_.loc[Patient_info_['subject_id'] == i,'neuropheno_id']  = _Patient_info_.loc[_Patient_info_['subject_id'] == i,'neuropheno_id']


In [ ]:
Patient_info_ph.columns

In [ ]:
for i in output['subject_id'].unique():
    output.loc[output['subject_id'] == i,'DoB'] = pd.to_datetime(Patient_info_.loc[Patient_info_['subject_id'] == i,'dob'].item())
    output.loc[output['subject_id'] == i,'DoB'] = pd.to_datetime(Patient_info_.loc[Patient_info_['subject_id'] == i,'dob'].item())



In [ ]:
output[['subject_id','Bars', 'Bars_Speech', 'Samples_no', 'Diagnosis','Prob_AT_tf_grad', 'Bars_Speech_pred_t','Bars_Total_pred_t']].head(200)

In [ ]:
output = output[['Date', 'subject_id', 'Neuropheno_ID', 'Bars', 'Bars_Speech', 'Sex',
       'Diagnosis', 'Samples_no', 'Prob_AT_tf_grad',
       'Bars_Speech_pred_t', 'Bars_Total_pred_t']]

output.to_csv('/home/kvattis/Desktop/All_dara_outputs_2022_05_03.csv', index = False)

In [ ]:
output_sca1 = output[output['Diagnosis'] != 0]
#output_sca2 = output[output['Diagnosis'] == 2]
#output_sca3 = output[output['Diagnosis'] == 3]
output_c = output[output['Diagnosis'] == 0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(output_c['Bars_Speech'], output_c['Prob_AT_tf_grad'], c = 'b' , marker = 'x')
ax.scatter(output_sca1['Bars_Speech'], output_sca1['Prob_AT_tf_grad'], c = 'k' , marker = 'x')
#ax.scatter(output_sca2['Bars_Speech'], output_sca2['Prob_AT_tf_grad'], c = 'g' , marker = 'x')
#ax.scatter(output_sca3['Bars_Speech'], output_sca3['Prob_AT_tf_grad'], c = 'r' , marker = 'x')
plt.xlim([-0.2, 4])
plt.ylim([-0.05, 1.05])

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
plt.plot([0,4],[0,4])

ax.scatter(output_sca1['Bars_Speech'], output_sca1['Bars_Speech_pred_t'], c = 'k' , marker = 'x')
#ax.scatter(output_sca2['Bars_Speech'], output_sca2['Bars_Speech_pred_t'], c = 'g' , marker = 'x')
#ax.scatter(output_sca3['Bars_Speech'], output_sca3['Bars_Speech_pred_t'], c = 'r' , marker = 'x')
ax.scatter(output_c['Bars_Speech'], output_c['Bars_Speech_pred_t'], c = 'b' , marker = 'x')

plt.xlim([-0.2, 4])
plt.ylim([-0.05, 4])

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
plt.plot([0,30],[0,30])
ax.scatter(output_c['Bars'], output_c['Bars_Total_pred_t'], c = 'b' , marker = 'x')
ax.scatter(output_sca1['Bars'], output_sca1['Bars_Total_pred_t'], c = 'k' , marker = 'x')
#ax.scatter(output_sca2['Bars'], output_sca2['Bars_Total_pred_t'], c = 'g' , marker = 'x')
#ax.scatter(output_sca3['Bars'], output_sca3['Bars_Total_pred_t'], c = 'r' , marker = 'x')
plt.xlim([-0.2, 30])
plt.ylim([-0.05, 30])

In [ ]:
Patient_info = pd.read_excel(data_home_dir + 'Subjects_BiogenSCA_with_PROM_2022_03_29_converted.xlsx',engine='openpyxl')

In [ ]:
Patient_info.columns